In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 93% 80.0M/85.7M [00:01<00:00, 68.8MB/s]
100% 85.7M/85.7M [00:01<00:00, 77.1MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 10.1 MB/s 
     |████████████████████████████████| 840 kB 55.7 MB/s 
     |████████████████████████████████| 120 kB 72.2 MB/s 
     |████████████████████████████████| 3.4 MB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 61.0 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.4 MB 59.2 MB/s 
     |████████████████████████████████| 87 kB 8.6 MB/s 
     |████████████████████████████████| 234 kB 63.4 MB/s 
     |████████████████████████████████| 1.2 MB 79.1 MB/s 
     |████████████████████████████████| 596 kB 85.4 MB/s 
     |████████████████████████████████| 1.1 MB 83.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.8 MB/s 
     |████████████████████████████████| 352 kB 73.7 MB/s 
     |████████████████████████████████| 47.7 MB 64.9 MB/s 
     |████████████████████████████████| 462 kB 75.0 MB/s 
     |████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [10]:
spec = model_spec.get('efficientdet_lite2')

In [11]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/10
397/397 [==============================] - 173s 321ms/step - det_loss: 1.0619 - cls_loss: 0.8233 - box_loss: 0.0048 - reg_l2_loss: 0.0771 - loss: 1.1390 - learning_rate: 0.0078 - gradient_norm: 3.6364 - val_det_loss: 0.6146 - val_cls_loss: 0.5433 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0774 - val_loss: 0.6921
Epoch 2/10
397/397 [==============================] - 124s 313ms/step - det_loss: 0.6482 - cls_loss: 0.5198 - box_loss: 0.0026 - reg_l2_loss: 0.0776 - loss: 0.7259 - learning_rate: 0.0070 - gradient_norm: 3.4913 - val_det_loss: 0.5013 - val_cls_loss: 0.4524 - val_box_loss: 9.7897e-04 - val_reg_l2_loss: 0.0779 - val_loss: 0.5792
Epoch 3/10
397/397 [==============================] - 124s 313ms/step - det_loss: 0.5678 - cls_loss: 0.4677 - box_loss: 0.0020 - reg_l2_loss: 0.0780 - loss: 0.6458 - learning_rate: 0.0061 - gradient_norm: 3.1580 - val_det_loss: 0.4487 - val_cls_loss: 0.3976 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.0781 - val_loss: 0.5268
Epoch 4/10
397/397

In [12]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 1354.0227739810944


In [13]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 10s 85ms/step

{'AP': 0.64008343, 'AP50': 0.77762264, 'AP75': 0.7176023, 'APs': -1.0, 'APm': 0.33791107, 'APl': 0.6580155, 'ARmax1': 0.6812883, 'ARmax10': 0.8062028, 'ARmax100': 0.81453186, 'ARs': -1.0, 'ARm': 0.5940476, 'ARl': 0.82824963, 'AP_/10': 0.5537684, 'AP_/20': 0.6717521, 'AP_/50': 0.508648, 'AP_/100': 0.77997935, 'AP_/200': 0.64832777, 'AP_/500': 0.66529983, 'AP_/2000': 0.65280867}


In [14]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 13.036428213119507


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [15]:
model.export(export_dir='/content/')